# Day 2 - PyTorch ANN (Implement ANN using Pytorch)

In [62]:
import pandas as pd 

from sklearn.datasets import load_iris 

iris = load_iris()

X = iris.data 
y = iris.target

y


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [6]:
from sklearn.model_selection import train_test_split 

X_train , X_test , y_train , y_test = train_test_split(X , y , random_state = 42 , test_size = 0.2)

In [7]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
X_train_scale = scale.fit_transform(X_train) 
X_test_scale  = scale.transform(X_test)


In [8]:
import torch 
from torch.utils.data import Dataset , DataLoader 



In [68]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):


    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
   
        x = self.data[idx]
        y = self.labels[idx]

        if self.transform:
            x = self.transform(x)

        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long) 

        return x, y


In [73]:
train_data = CustomDataset(X_train , y_train , transform = None)
test_data = CustomDataset(X_test , y_test , transform = None)

In [74]:
train_dataloader = DataLoader(train_data , shuffle = True , batch_size = 32) 
test_dataloader  = DataLoader(test_data , shuffle = False  , batch_size = 32)

In [75]:
for X_batch , y_batch in train_dataloader: 
    print("train Batch X shape:", X_batch.shape)
    print("train Batch y shape:", y_batch.shape)
    break

for X_batch , y_batch in test_dataloader: 
    print("test Batch X shape:", X_batch.shape)
    print("test Batch y shape:", y_batch.shape)
    break

train Batch X shape: torch.Size([32, 4])
train Batch y shape: torch.Size([32])
test Batch X shape: torch.Size([30, 4])
test Batch y shape: torch.Size([30])


In [86]:
# creating ANN 


import torch.nn as nn 

class ANN(nn.Module): 
    def __init__(self , input_dim ):
        super(ANN , self).__init__()

        self.model = nn.Sequential(
            nn.Linear(in_features = input_dim , out_features = 10) , 
            nn.BatchNorm1d(10) , 
            nn.ReLU() , 
            nn.Dropout(p = 0.3) , 

            nn.Linear(in_features = 10 , out_features = 10) , 
            nn.BatchNorm1d(10) , 
            nn.ReLU() , 
            nn.Dropout(p = 0.3) , 

            nn.Linear(in_features = 10 , out_features = 3)             

        )

    def forward(self , x): 
        return self.model(x)
    

In [87]:
Model = ANN(input_dim = 4)
Model.parameters

<bound method Module.parameters of ANN(
  (model): Sequential(
    (0): Linear(in_features=4, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=10, out_features=3, bias=True)
  )
)>

In [88]:
epochs = 1000
lr = 0.01 
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(Model.parameters() , lr = lr , momentum = 0.4)


In [89]:
# training and testing loop 


for epoch in range(epochs): 

    # set to training 
    Model.train() 

    train_loss = 0.0
    test_loss = 0.0

    for X_batch , y_batch in train_dataloader : 

        outputs = Model(X_batch)

        loss = loss_fn(outputs , y_batch)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        train_loss += loss.item()

    Model.eval()
    with torch.inference_mode():
        for X_batch , y_batch in test_dataloader : 
    
            outputs = Model(X_batch)
    
            loss = loss_fn(outputs , y_batch)
    
            test_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataloader)
    avg_tets_loss = test_loss / len(test_dataloader)

    if epoch % 100 == 0 : 
        print(f" Epoch : {epoch} , Train_loss : {avg_train_loss : .4f} , test_loss : {avg_tets_loss : .4f} ")
        



 Epoch : 0 , Train_loss :  1.1466 , test_loss :  1.0800 
 Epoch : 100 , Train_loss :  0.4622 , test_loss :  0.2784 
 Epoch : 200 , Train_loss :  0.3332 , test_loss :  0.1852 
 Epoch : 300 , Train_loss :  0.3420 , test_loss :  0.1557 
 Epoch : 400 , Train_loss :  0.2870 , test_loss :  0.1126 
 Epoch : 500 , Train_loss :  0.3268 , test_loss :  0.1006 
 Epoch : 600 , Train_loss :  0.2727 , test_loss :  0.1122 
 Epoch : 700 , Train_loss :  0.2931 , test_loss :  0.0853 
 Epoch : 800 , Train_loss :  0.1676 , test_loss :  0.0804 
 Epoch : 900 , Train_loss :  0.1965 , test_loss :  0.0857 


In [93]:
import numpy as np 
X_new = np.array([
    [5.1, 3.5, 1.4, 0.2],  
    [6.0, 2.2, 4.0, 1.0],  
    [6.3, 3.3, 6.0, 2.5]   # should map to class 2
])
X_new_scaled = scale.transform(X_new)

X_new_tensor = torch.tensor(X_new_scaled, dtype=torch.float32)

Model.eval()
with torch.no_grad():
    outputs = Model(X_new_tensor)
    predictions = torch.argmax(outputs, dim=1)

print("Predicted classes:", predictions.numpy())


Predicted classes: [0 0 1]
